In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import tensorflow as tf
import time

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

In [2]:
# load data: digits 5 to 9, but still label with 0 to 4, 
# because TensorFlow expects label's integers from 0 to n_classes-1.
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

X_train2_full = mnist.train.images[mnist.train.labels >= 5]
y_train2_full = mnist.train.labels[mnist.train.labels >= 5] - 5
X_valid2_full = mnist.validation.images[mnist.validation.labels >= 5]
y_valid2_full = mnist.validation.labels[mnist.validation.labels >= 5] - 5
X_test2 = mnist.test.images[mnist.test.labels >= 5]
y_test2 = mnist.test.labels[mnist.test.labels >= 5] - 5

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
# we want to keep only 100 instances per class in the training set 
# and let's keep only 30 instances per class in the validation set
# tesing set is already loaded above
def sample_n_instances_per_class(X, y, n=100):
    Xs, ys = [], []
    for label in np.unique(y):
        idx = (y == label)
        Xc = X[idx][:n]
        yc = y[idx][:n]
        Xs.append(Xc)
        ys.append(yc)
    return np.concatenate(Xs), np.concatenate(ys)

X_train2, y_train2 = sample_n_instances_per_class(X_train2_full, y_train2_full, n=100)
X_valid2, y_valid2 = sample_n_instances_per_class(X_valid2_full, y_valid2_full, n=30)

## 3.1 Train Softmax Only

In [4]:
reset_graph() # clean the graph

"""
Graph Define Step
"""
restore_saver = tf.train.import_meta_graph("./Models/Team26_HW2.ckpt.meta") # load meta graph

"""
1. Restore the variable about the component in graph which help to retrieve and assign value
"""
graph = tf.get_default_graph() # get original graph
X = graph.get_tensor_by_name("Input_X:0") # assign placeholder variables with input X
y = graph.get_tensor_by_name("Input_Label:0")# assign placeholder variables with input Label
loss = graph.get_tensor_by_name("Loss:0") # assign variables about loss
Y_proba = graph.get_tensor_by_name("Softmax:0") # assign variables about loss
logits = Y_proba.op.inputs[0] # assign variables about logits
accuracy = graph.get_tensor_by_name("Accuracy:0") # assign variables  about accuracy

"""
2. Get the logits layer which is what we would like to train
"""
training_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="Logits") # get logits layer by scope name

"""
3. Define the optimizer and assign to only update on softmax_layer which defined above
"""
optimizer = tf.train.AdamOptimizer(0.0017, name="Adam_Op") # create a new optimizer
training_op = optimizer.minimize(loss, var_list=training_layer_vars) # minimize the loss and only update certern variables

var_list_init = tf.variables_initializer([optimizer.get_slot(var, name) for name in optimizer.get_slot_names() for var in training_layer_vars]+list(optimizer._get_beta_accumulators())) # to get the variables name about the logits layer 
"""
Training preparation
"""

no_update_tolerant = 20 # set the criteria that if the model didn't get better for how many epochs

no_update_progress = 0 # the variables to record for the not better epochs
current_best_accuracy = 0.0 # variables to record the best valid accuracy
current_best_loss = 10.0 # variables to record the best valid loss

save_count = 0 # initial the saver counter
saver = tf.train.Saver() # to store the model

with tf.Session() as sess: # start the tensorflow nn session
    sess.run(tf.global_variables_initializer()) # initialize all the parameter
    restore_saver.restore(sess, './Models/Team26_HW2.ckpt') # restore pretrain weights

    """
    Variables Initialization
    """
    sess.run(var_list_init) # initialize the un-initizlized variables
    """
    Training Step
    """
    begin_time = time.time() # record the begin training time
    for epochs in range(1000): # training on 1000 epochs
        _, train_loss = sess.run([training_op, loss], feed_dict={X:X_train2, y:y_train2}) # feed with training variables
        valid_loss, valid_accuracy = sess.run([loss, accuracy], feed_dict={X:X_valid2, y:y_valid2}) # feed with validation variables
        test_loss ,  test_accuracy = sess.run([loss, accuracy], feed_dict={X:X_test2, y:y_test2}) # feed with testing variables
        
        if valid_accuracy > current_best_accuracy: # if better
            current_best_accuracy = valid_accuracy # keep record
            save_path = saver.save(sess, "./Models/TransferLearning/HW3_1/Team26_HW3_1.ckpt") #keep the best model
            save_count +=1 # add recorder for save count
            print("-------\nBest Model Found! Save! Keep Count:{:3d}".format(save_count)) # notification if save the model
            no_update_progress = 0 # reset no update progress
        else:                                      # if not better
            no_update_progress += 1                # record on no update

        if valid_loss < current_best_loss: # if better
            current_best_loss = valid_loss # keep record       
            
        if no_update_progress == no_update_tolerant: break # stop training
            
                
        print("{:4d} epochs,   Validation loss:{:.5f},   Best loss:{:.5f},   Accuracy:{:.2f}%/ {:.2f}%".format(
            epochs+1, # show current epochs
            valid_loss, # show validation loss
            current_best_loss, # showe best loss                                                                       
            valid_accuracy*100, # to show as % we need to *100
            test_accuracy*100 # to show as % we need to *100
        )) # print the performane for each epochs
    finish_itme = time.time() # record the finish time
    print("\nTraining Time:{:3f}, Total Save {:3d} Times".format(finish_itme-begin_time, save_count)) # calculate and print training time
        
        

INFO:tensorflow:Restoring parameters from ./Models/Team26_HW2.ckpt
-------
Best Model Found! Save! Keep Count:  1
   1 epochs,   Validation loss:5.78928,   Best loss:5.78928,   Accuracy:37.33%/ 39.44%
-------
Best Model Found! Save! Keep Count:  2
   2 epochs,   Validation loss:5.61240,   Best loss:5.61240,   Accuracy:38.00%/ 39.77%
-------
Best Model Found! Save! Keep Count:  3
   3 epochs,   Validation loss:5.43892,   Best loss:5.43892,   Accuracy:38.67%/ 40.34%
   4 epochs,   Validation loss:5.26864,   Best loss:5.26864,   Accuracy:38.67%/ 40.73%
-------
Best Model Found! Save! Keep Count:  4
   5 epochs,   Validation loss:5.10148,   Best loss:5.10148,   Accuracy:39.33%/ 41.23%
-------
Best Model Found! Save! Keep Count:  5
   6 epochs,   Validation loss:4.93752,   Best loss:4.93752,   Accuracy:40.67%/ 41.70%
-------
Best Model Found! Save! Keep Count:  6
   7 epochs,   Validation loss:4.77690,   Best loss:4.77690,   Accuracy:40.67%/ 42.01%
-------
Best Model Found! Save! Keep Count

-------
Best Model Found! Save! Keep Count: 40
  76 epochs,   Validation loss:0.95635,   Best loss:0.95635,   Accuracy:78.00%/ 72.04%
  77 epochs,   Validation loss:0.94861,   Best loss:0.94861,   Accuracy:78.00%/ 72.31%
  78 epochs,   Validation loss:0.94096,   Best loss:0.94096,   Accuracy:78.00%/ 72.43%
  79 epochs,   Validation loss:0.93338,   Best loss:0.93338,   Accuracy:78.00%/ 72.58%
  80 epochs,   Validation loss:0.92585,   Best loss:0.92585,   Accuracy:78.00%/ 72.78%
  81 epochs,   Validation loss:0.91837,   Best loss:0.91837,   Accuracy:78.00%/ 72.99%
  82 epochs,   Validation loss:0.91093,   Best loss:0.91093,   Accuracy:78.00%/ 73.22%
  83 epochs,   Validation loss:0.90351,   Best loss:0.90351,   Accuracy:78.00%/ 73.42%
  84 epochs,   Validation loss:0.89613,   Best loss:0.89613,   Accuracy:78.00%/ 73.57%
  85 epochs,   Validation loss:0.88878,   Best loss:0.88878,   Accuracy:78.00%/ 73.77%
  86 epochs,   Validation loss:0.88146,   Best loss:0.88146,   Accuracy:78.00%/ 74.

## 3.2 Cache the Output of FIfth layer for Speed Up

In [5]:
reset_graph() # clean the graph

"""
Graph Define Step
"""
restore_saver = tf.train.import_meta_graph("./Models/Team26_HW2.ckpt.meta") # load meta graph
   
""" 
1. Restore the variable about the component in graph which help to retrieve and assign value
"""
graph = tf.get_default_graph() # get original graph
X = graph.get_tensor_by_name("Input_X:0") # assign placeholder variables with input X
y = graph.get_tensor_by_name("Input_Label:0")# assign placeholder variables with input Label
loss = graph.get_tensor_by_name("Loss:0") # assign variables about loss
Y_proba = graph.get_tensor_by_name("Softmax:0") # assign variables about loss
logits = Y_proba.op.inputs[0] # assign variables about logits
accuracy = graph.get_tensor_by_name("Accuracy:0") # assign variables  about accuracy

fifth_layer_out = graph.get_tensor_by_name("Fully_Connected_Layer_5/Elu:0").op.outputs[0] # get the output tensors of fifth's layers

"""
2. Get the softmax layer which is what we would like to train
"""
training_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="Logits") # get logits layer by scope name

"""
3. Define the optimizer and assign to only update on softmax_layer which defined above
"""
optimizer = tf.train.AdamOptimizer(0.0017, name="Adam_Op") # create a new optimizer
training_op = optimizer.minimize(loss, var_list=training_layer_vars) # minimize the loss and only update certern variables

var_list_init = tf.variables_initializer([optimizer.get_slot(var, name) for name in optimizer.get_slot_names() for var in training_layer_vars]+list(optimizer._get_beta_accumulators()))# to get the variables name about the logits layer 
"""
Training preparation
"""

no_update_tolerant = 20 # set the criteria that if the model didn't get better for how many epochs

no_update_progress = 0 # the variables to record for the not better epochs
current_best_accuracy = 0.0 # variables to record the best valid accuracy
current_best_loss = 10.0 # variables to record the best valid loss

save_count = 0 # initial the saver counter
saver = tf.train.Saver() # to store the model

with tf.Session() as sess: # start the tensorflow nn session
    sess.run(tf.global_variables_initializer()) # initialize all the parameter
    restore_saver.restore(sess, './Models/Team26_HW2.ckpt') # restore pretrain weights
   
    """
    Variables Initialization
    """
    sess.run(var_list_init) # initialize the un-initialized value
    
    begin_time = time.time() # record the begin training time
    """
    Get the training output of fifth layer
    """
    fifth_layer_out_cache = sess.run(fifth_layer_out, feed_dict={X:X_train2, y:y_train2}) # feed in the training value and get the fifth layer value out
    fifth_layer_validation_out_cache = sess.run(fifth_layer_out, feed_dict={X:X_valid2, y:y_valid2}) # feed in the validation value and get the fifth layer value out
    
    """
    Training Step
    """
    
    for epochs in range(1000): # training on 1000 epochs
        _, train_loss = sess.run([training_op, loss], feed_dict={fifth_layer_out:fifth_layer_out_cache, y:y_train2}) # feed with training variables
        valid_loss, valid_accuracy = sess.run([loss, accuracy], feed_dict={fifth_layer_out:fifth_layer_validation_out_cache, y:y_valid2}) # feed with validation variables
        test_loss ,  test_accuracy = sess.run([loss, accuracy], feed_dict={X:X_test2, y:y_test2}) # feed with testing variables
        
        if valid_accuracy > current_best_accuracy: # if better
            current_best_accuracy = valid_accuracy # keep record
            save_path = saver.save(sess, "./Models/TransferLearning/HW3_2/Team26_HW3_2.ckpt") #keep the best model
            save_count +=1 # add recorder for save count
            print("-------\nBest Model Found! Save! Keep Count:{:3d}".format(save_count)) # notification if save the model
            no_update_progress = 0 # reset no update progress
        else:                                      # if not better
            no_update_progress += 1                # record on no update

        if valid_loss < current_best_loss: # if better
            current_best_loss = valid_loss # keep record            
            
        if no_update_progress == no_update_tolerant: break # stop training
            
                
        print("{:4d} epochs,   Validation loss:{:.5f},   Best loss:{:.5f},   Accuracy:{:.2f}%/ {:.2f}%".format(
            epochs+1, # show current epochs
            valid_loss, # show validation loss
            current_best_loss, # showe best loss                                                                       
            valid_accuracy*100, # to show as % we need to *100
            test_accuracy*100 # to show as % we need to *100
        )) # print the performane for each epochs
    finish_itme = time.time() # record the finish time
    print("\nTraining Time:{:3f}, Total Save {:3d} Times".format(finish_itme-begin_time, save_count)) # calculate and print training time
        
        

INFO:tensorflow:Restoring parameters from ./Models/Team26_HW2.ckpt
-------
Best Model Found! Save! Keep Count:  1
   1 epochs,   Validation loss:5.78928,   Best loss:5.78928,   Accuracy:37.33%/ 39.44%
-------
Best Model Found! Save! Keep Count:  2
   2 epochs,   Validation loss:5.61240,   Best loss:5.61240,   Accuracy:38.00%/ 39.77%
-------
Best Model Found! Save! Keep Count:  3
   3 epochs,   Validation loss:5.43892,   Best loss:5.43892,   Accuracy:38.67%/ 40.34%
   4 epochs,   Validation loss:5.26864,   Best loss:5.26864,   Accuracy:38.67%/ 40.73%
-------
Best Model Found! Save! Keep Count:  4
   5 epochs,   Validation loss:5.10148,   Best loss:5.10148,   Accuracy:39.33%/ 41.23%
-------
Best Model Found! Save! Keep Count:  5
   6 epochs,   Validation loss:4.93752,   Best loss:4.93752,   Accuracy:40.67%/ 41.70%
   7 epochs,   Validation loss:4.77690,   Best loss:4.77690,   Accuracy:40.67%/ 42.01%
-------
Best Model Found! Save! Keep Count:  6
   8 epochs,   Validation loss:4.61978,   

-------
Best Model Found! Save! Keep Count: 41
  76 epochs,   Validation loss:0.95635,   Best loss:0.95635,   Accuracy:78.00%/ 72.04%
  77 epochs,   Validation loss:0.94861,   Best loss:0.94861,   Accuracy:78.00%/ 72.31%
  78 epochs,   Validation loss:0.94096,   Best loss:0.94096,   Accuracy:78.00%/ 72.43%
  79 epochs,   Validation loss:0.93338,   Best loss:0.93338,   Accuracy:78.00%/ 72.58%
  80 epochs,   Validation loss:0.92585,   Best loss:0.92585,   Accuracy:78.00%/ 72.78%
  81 epochs,   Validation loss:0.91837,   Best loss:0.91837,   Accuracy:78.00%/ 72.99%
  82 epochs,   Validation loss:0.91093,   Best loss:0.91093,   Accuracy:78.00%/ 73.22%
  83 epochs,   Validation loss:0.90351,   Best loss:0.90351,   Accuracy:78.00%/ 73.42%
  84 epochs,   Validation loss:0.89613,   Best loss:0.89613,   Accuracy:78.00%/ 73.57%
  85 epochs,   Validation loss:0.88878,   Best loss:0.88878,   Accuracy:78.00%/ 73.77%
  86 epochs,   Validation loss:0.88146,   Best loss:0.88146,   Accuracy:78.00%/ 74.

Because of the difference of times that we save the model, cache may not faster then non-cache one.

If we remove the saver, the cache is definitely faster then non-cache one.

The experiments shows below

### 3.2.1 No Saver without Cache

In [9]:
reset_graph() # clean the graph

"""
Graph Define Step
"""
restore_saver = tf.train.import_meta_graph("./Models/Team26_HW2.ckpt.meta") # load meta graph

"""
1. Restore the variable about the component in graph which help to retrieve and assign value
"""
graph = tf.get_default_graph() # get original graph
X = graph.get_tensor_by_name("Input_X:0") # assign placeholder variables with input X
y = graph.get_tensor_by_name("Input_Label:0")# assign placeholder variables with input Label
loss = graph.get_tensor_by_name("Loss:0") # assign variables about loss
Y_proba = graph.get_tensor_by_name("Softmax:0") # assign variables about loss
logits = Y_proba.op.inputs[0] # assign variables about logits
accuracy = graph.get_tensor_by_name("Accuracy:0") # assign variables  about accuracy

"""
2. Get the logits layer which is what we would like to train
"""
training_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="Logits") # get logits layer by scope name

"""
3. Define the optimizer and assign to only update on softmax_layer which defined above
"""
optimizer = tf.train.AdamOptimizer(0.0017, name="Adam_Op") # create a new optimizer
training_op = optimizer.minimize(loss, var_list=training_layer_vars) # minimize the loss and only update certern variables

var_list_init = tf.variables_initializer([optimizer.get_slot(var, name) for name in optimizer.get_slot_names() for var in training_layer_vars]+list(optimizer._get_beta_accumulators())) # to get the variables name about the logits layer 
"""
Training preparation
"""

no_update_tolerant = 20 # set the criteria that if the model didn't get better for how many epochs

no_update_progress = 0 # the variables to record for the not better epochs
current_best_accuracy = 0.0 # variables to record the best valid accuracy
current_best_loss = 10.0 # variables to record the best valid loss

# save_count = 0 # initial the saver counter
# saver = tf.train.Saver() # to store the model

with tf.Session() as sess: # start the tensorflow nn session
    sess.run(tf.global_variables_initializer()) # initialize all the parameter
    restore_saver.restore(sess, './Models/Team26_HW2.ckpt') # restore pretrain weights

    """
    Variables Initialization
    """
    sess.run(var_list_init) # initialize the un-initizlized variables
    """
    Training Step
    """
    begin_time = time.time() # record the begin training time
    for epochs in range(1000): # training on 1000 epochs
        _, train_loss = sess.run([training_op, loss], feed_dict={X:X_train2, y:y_train2}) # feed with training variables
        valid_loss, valid_accuracy = sess.run([loss, accuracy], feed_dict={X:X_valid2, y:y_valid2}) # feed with validation variables
        test_loss ,  test_accuracy = sess.run([loss, accuracy], feed_dict={X:X_test2, y:y_test2}) # feed with testing variables
        
        if valid_accuracy > current_best_accuracy: # if better
            current_best_accuracy = valid_accuracy # keep record
#             save_path = saver.save(sess, "./Models/TransferLearning/HW3_1/Team26_HW3_1.ckpt") #keep the best model
#             save_count +=1 # add recorder for save count
#             print("-------\nBest Model Found! Save! Keep Count:{:3d}".format(save_count)) # notification if save the model
            no_update_progress = 0 # reset no update progress
        else:                                      # if not better
            no_update_progress += 1                # record on no update

        if valid_loss < current_best_loss: # if better
            current_best_loss = valid_loss # keep record       
            
        if no_update_progress == no_update_tolerant: break # stop training
            
                
        print("{:4d} epochs,   Validation loss:{:.5f},   Best loss:{:.5f},   Accuracy:{:.2f}%/ {:.2f}%".format(
            epochs+1, # show current epochs
            valid_loss, # show validation loss
            current_best_loss, # showe best loss                                                                       
            valid_accuracy*100, # to show as % we need to *100
            test_accuracy*100 # to show as % we need to *100
        )) # print the performane for each epochs
    finish_itme = time.time() # record the finish time
    print("\nTraining Time:{:3f}".format(finish_itme-begin_time)) # calculate and print training time
        
        

INFO:tensorflow:Restoring parameters from ./Models/Team26_HW2.ckpt
   1 epochs,   Validation loss:5.78928,   Best loss:5.78928,   Accuracy:37.33%/ 39.44%
   2 epochs,   Validation loss:5.61240,   Best loss:5.61240,   Accuracy:38.00%/ 39.77%
   3 epochs,   Validation loss:5.43892,   Best loss:5.43892,   Accuracy:38.67%/ 40.34%
   4 epochs,   Validation loss:5.26864,   Best loss:5.26864,   Accuracy:38.67%/ 40.73%
   5 epochs,   Validation loss:5.10148,   Best loss:5.10148,   Accuracy:39.33%/ 41.23%
   6 epochs,   Validation loss:4.93752,   Best loss:4.93752,   Accuracy:40.67%/ 41.70%
   7 epochs,   Validation loss:4.77690,   Best loss:4.77690,   Accuracy:40.67%/ 42.01%
   8 epochs,   Validation loss:4.61978,   Best loss:4.61978,   Accuracy:41.33%/ 42.50%
   9 epochs,   Validation loss:4.46633,   Best loss:4.46633,   Accuracy:42.67%/ 42.75%
  10 epochs,   Validation loss:4.31674,   Best loss:4.31674,   Accuracy:42.67%/ 43.22%
  11 epochs,   Validation loss:4.17121,   Best loss:4.17121,   

 107 epochs,   Validation loss:0.74390,   Best loss:0.74390,   Accuracy:81.33%/ 76.98%
 108 epochs,   Validation loss:0.73828,   Best loss:0.73828,   Accuracy:82.00%/ 77.19%
 109 epochs,   Validation loss:0.73273,   Best loss:0.73273,   Accuracy:82.00%/ 77.31%
 110 epochs,   Validation loss:0.72727,   Best loss:0.72727,   Accuracy:82.00%/ 77.37%

Training Time:0.594234


### 3.2. No Saver with Cache

In [8]:
reset_graph() # clean the graph

"""
Graph Define Step
"""
restore_saver = tf.train.import_meta_graph("./Models/Team26_HW2.ckpt.meta") # load meta graph
   
""" 
1. Restore the variable about the component in graph which help to retrieve and assign value
"""
graph = tf.get_default_graph() # get original graph
X = graph.get_tensor_by_name("Input_X:0") # assign placeholder variables with input X
y = graph.get_tensor_by_name("Input_Label:0")# assign placeholder variables with input Label
loss = graph.get_tensor_by_name("Loss:0") # assign variables about loss
Y_proba = graph.get_tensor_by_name("Softmax:0") # assign variables about loss
logits = Y_proba.op.inputs[0] # assign variables about logits
accuracy = graph.get_tensor_by_name("Accuracy:0") # assign variables  about accuracy

fifth_layer_out = graph.get_tensor_by_name("Fully_Connected_Layer_5/Elu:0").op.outputs[0] # get the output tensors of fifth's layers

"""
2. Get the softmax layer which is what we would like to train
"""
training_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="Logits") # get logits layer by scope name

"""
3. Define the optimizer and assign to only update on softmax_layer which defined above
"""
optimizer = tf.train.AdamOptimizer(0.0017, name="Adam_Op") # create a new optimizer
training_op = optimizer.minimize(loss, var_list=training_layer_vars) # minimize the loss and only update certern variables

var_list_init = tf.variables_initializer([optimizer.get_slot(var, name) for name in optimizer.get_slot_names() for var in training_layer_vars]+list(optimizer._get_beta_accumulators()))# to get the variables name about the logits layer 
"""
Training preparation
"""

no_update_tolerant = 20 # set the criteria that if the model didn't get better for how many epochs

no_update_progress = 0 # the variables to record for the not better epochs
current_best_accuracy = 0.0 # variables to record the best valid accuracy
current_best_loss = 10.0 # variables to record the best valid loss

# save_count = 0 # initial the saver counter
# saver = tf.train.Saver() # to store the model

with tf.Session() as sess: # start the tensorflow nn session
    sess.run(tf.global_variables_initializer()) # initialize all the parameter
    restore_saver.restore(sess, './Models/Team26_HW2.ckpt') # restore pretrain weights
   
    """
    Variables Initialization
    """
    sess.run(var_list_init) # initialize the un-initialized value
    
    begin_time = time.time() # record the begin training time
    """
    Get the training output of fifth layer
    """
    fifth_layer_out_cache = sess.run(fifth_layer_out, feed_dict={X:X_train2, y:y_train2}) # feed in the training value and get the fifth layer value out
    fifth_layer_validation_out_cache = sess.run(fifth_layer_out, feed_dict={X:X_valid2, y:y_valid2}) # feed in the validation value and get the fifth layer value out
    
    """
    Training Step
    """
    
    for epochs in range(1000): # training on 1000 epochs
        _, train_loss = sess.run([training_op, loss], feed_dict={fifth_layer_out:fifth_layer_out_cache, y:y_train2}) # feed with training variables
        valid_loss, valid_accuracy = sess.run([loss, accuracy], feed_dict={fifth_layer_out:fifth_layer_validation_out_cache, y:y_valid2}) # feed with validation variables
        test_loss ,  test_accuracy = sess.run([loss, accuracy], feed_dict={X:X_test2, y:y_test2}) # feed with testing variables
        
        if valid_accuracy > current_best_accuracy: # if better
            current_best_accuracy = valid_accuracy # keep record
#             save_path = saver.save(sess, "./Models/TransferLearning/HW3_2/Team26_HW3_2.ckpt") #keep the best model
#             save_count +=1 # add recorder for save count
#             print("-------\nBest Model Found! Save! Keep Count:{:3d}".format(save_count)) # notification if save the model
            no_update_progress = 0 # reset no update progress
        else:                                      # if not better
            no_update_progress += 1                # record on no update

        if valid_loss < current_best_loss: # if better
            current_best_loss = valid_loss # keep record            
            
        if no_update_progress == no_update_tolerant: break # stop training
            
                
        print("{:4d} epochs,   Validation loss:{:.5f},   Best loss:{:.5f},   Accuracy:{:.2f}%/ {:.2f}%".format(
            epochs+1, # show current epochs
            valid_loss, # show validation loss
            current_best_loss, # showe best loss                                                                       
            valid_accuracy*100, # to show as % we need to *100
            test_accuracy*100 # to show as % we need to *100
        )) # print the performane for each epochs
    finish_itme = time.time() # record the finish time
    print("\nTraining Time:{:3f}".format(finish_itme-begin_time)) # calculate and print training time
        
        

INFO:tensorflow:Restoring parameters from ./Models/Team26_HW2.ckpt
   1 epochs,   Validation loss:5.78928,   Best loss:5.78928,   Accuracy:37.33%/ 39.44%
   2 epochs,   Validation loss:5.61240,   Best loss:5.61240,   Accuracy:38.00%/ 39.77%
   3 epochs,   Validation loss:5.43892,   Best loss:5.43892,   Accuracy:38.67%/ 40.34%
   4 epochs,   Validation loss:5.26864,   Best loss:5.26864,   Accuracy:38.67%/ 40.73%
   5 epochs,   Validation loss:5.10148,   Best loss:5.10148,   Accuracy:39.33%/ 41.23%
   6 epochs,   Validation loss:4.93752,   Best loss:4.93752,   Accuracy:40.67%/ 41.70%
   7 epochs,   Validation loss:4.77690,   Best loss:4.77690,   Accuracy:40.67%/ 42.01%
   8 epochs,   Validation loss:4.61978,   Best loss:4.61978,   Accuracy:41.33%/ 42.50%
   9 epochs,   Validation loss:4.46633,   Best loss:4.46633,   Accuracy:42.67%/ 42.75%
  10 epochs,   Validation loss:4.31674,   Best loss:4.31674,   Accuracy:42.67%/ 43.22%
  11 epochs,   Validation loss:4.17121,   Best loss:4.17121,   

## 3.3 4 Layers and Train on Softmax Layer

In [ ]:
reset_graph() # clean the graph

"""
Graph Define Step
"""
restore_saver = tf.train.import_meta_graph("./Models/Team26_HW2.ckpt.meta") # load meta graph
"""
1. Restore the variable about the component in graph which help to retrieve and assign value
"""
graph = tf.get_default_graph() # get original graph
X = graph.get_tensor_by_name("Input_X:0") # assign placeholder variables with input X
y = graph.get_tensor_by_name("Input_Label:0")# assign placeholder variables with input Label

fourth_layer_out = graph.get_tensor_by_name("Fully_Connected_Layer_4/Elu:0").op.outputs[0] # get the output of fourth layer

temp = set(tf.global_variables()) # record the the variables now, which will be used after new variables are created
"""
2.Create new logits, softmax layer and connect old fourth layer, connect to softmax layer
"""
logits = tf.layers.dense(inputs=fourth_layer_out, # assign fourth layer's output as input
                         units=5, # converge to five nodes
                         kernel_initializer=tf.contrib.layers.variance_scaling_initializer(), # initial with he initializer
                         name = "New_Logits" # name this layer as New_Logists
                        )  # Create the new logits layer using dense

y_ = tf.nn.softmax(logits, name="Y_proba") # Softmax Layer

"""
3. Get the logits layer which is what we would like to train
"""
training_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="New_Logits") # get new logits layer by scope name

"""
4. Define loss, optimizer and assign to only update on softmax_layer which defined above
"""   
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits, name="New_Cross_Entropy"), name="New_Loss") # Refine Loss Function

optimizer = tf.train.AdamOptimizer(0.0017, name="Adam_Op") # new adam optimizer
training_op = optimizer.minimize(loss, var_list=training_layer_vars) # minimize the loss and only update certern variables

"""
5. Define new evaluation metrics
"""
predicted_class = tf.argmax(y_, 1, output_type=tf.int32) # Define Accuracy
correct_predict = tf.equal(y, predicted_class) # [True, False ..., True]
accuracy = tf.reduce_mean(tf.cast(correct_predict, tf.float32)) # [True, False ..., True] --> [1,0,...,1]

var_list_init = tf.variables_initializer(set(tf.global_variables())-temp) # to get new variables that we created afterward
"""
Training preparation
"""

no_update_tolerant = 20 # set the criteria that if the model didn't get better for how many epochs

no_update_progress = 0 # the variables to record for the not better epochs
current_best_accuracy = 0.0 # variables to record the best valid accuracy
current_best_loss = 10.0 # variables to record the best valid loss

saver = tf.train.Saver() # to store the model

with tf.Session() as sess: # start the tensorflow nn session
    sess.run(tf.global_variables_initializer()) # initialize all the parameter
    restore_saver.restore(sess, './Models/Team26_HW2.ckpt') # restore pretrain weights
    
    """
    Initialize all the variables
    """
    sess.run(var_list_init) # init the variable about fourth + softmax layer + adam optimizer
    
    """
    Get the training output of fourth layer
    """
    fourth_layer_out_cache = sess.run(fourth_layer_out, feed_dict={X:X_train2, y:y_train2}) # get the output of fourth layer
    fourth_layer_out_valid_cache = sess.run(fourth_layer_out, feed_dict={X:X_valid2, y:y_train2}) # get the output of fourth layer

    
    """
    Training Step
    """
    begin_time = time.time()  # record the begin training time
    for epochs in range(1000): # training on 1000 epochs
        _, train_loss = sess.run([training_op, loss], feed_dict={fourth_layer_out:fourth_layer_out_cache, y:y_train2}) # feed with training variables
        valid_loss, valid_accuracy = sess.run([loss, accuracy], feed_dict={fourth_layer_out:fourth_layer_out_valid_cache, y:y_valid2}) # feed with validation variables
        test_loss ,  test_accuracy = sess.run([loss, accuracy], feed_dict={X:X_test2, y:y_test2}) # feed with testing variables
        
        if valid_accuracy > current_best_accuracy: # if better
            current_best_accuracy = valid_accuracy # keep record
            save_path = saver.save(sess, "./Models/TransferLearning/HW3_3/Team26_HW3_3.ckpt") #keep the best model
            no_update_progress = 0
        else:                                      # if not better
            no_update_progress += 1                # record on no update

        if valid_loss < current_best_loss: # if better
            current_best_loss = valid_loss # keep record            
            
        if no_update_progress == no_update_tolerant: break # stop training
            
                
        print("{:4d} epochs,   Validation loss:{:.5f},   Best loss:{:.5f},   Accuracy:{:.2f}%/ {:.2f}%".format(
            epochs+1, # show current epochs
            valid_loss, # show validation loss
            current_best_loss, # showe best loss                                                                       
            valid_accuracy*100, # to show as % we need to *100
            test_accuracy*100 # to show as % we need to *100
        )) # print the performane for each epochs
    finish_itme = time.time() # record the finish time
    print("\nTraining Time:{:3f}".format(finish_itme-begin_time)) # calculate and print training time
        
        

INFO:tensorflow:Restoring parameters from ./Models/Team26_HW2.ckpt
   1 epochs,   Validation loss:1.92721,   Best loss:1.92721,   Accuracy:25.33%/ 20.24%
   2 epochs,   Validation loss:1.82962,   Best loss:1.82962,   Accuracy:28.00%/ 22.38%
   3 epochs,   Validation loss:1.73744,   Best loss:1.73744,   Accuracy:29.33%/ 24.23%
   4 epochs,   Validation loss:1.65103,   Best loss:1.65103,   Accuracy:35.33%/ 26.56%
   5 epochs,   Validation loss:1.57051,   Best loss:1.57051,   Accuracy:38.00%/ 29.29%
   6 epochs,   Validation loss:1.49569,   Best loss:1.49569,   Accuracy:42.00%/ 31.99%
   7 epochs,   Validation loss:1.42635,   Best loss:1.42635,   Accuracy:43.33%/ 34.44%
   8 epochs,   Validation loss:1.36219,   Best loss:1.36219,   Accuracy:46.00%/ 37.34%
   9 epochs,   Validation loss:1.30289,   Best loss:1.30289,   Accuracy:46.67%/ 39.81%
  10 epochs,   Validation loss:1.24812,   Best loss:1.24812,   Accuracy:50.00%/ 42.32%
  11 epochs,   Validation loss:1.19753,   Best loss:1.19753,   

## 3.4 Only Frozen 3th & 4th Layers

In [ ]:
reset_graph() # clean the graph

"""
Graph Define Step
"""
restore_saver = tf.train.import_meta_graph("./Models/Team26_HW2.ckpt.meta") # load meta graph
"""
1. Restore the variable about the component in graph which help to retrieve and assign value
"""
graph = tf.get_default_graph() # get original graph
X = graph.get_tensor_by_name("Input_X:0") # assign placeholder variables with input X
y = graph.get_tensor_by_name("Input_Label:0")# assign placeholder variables with input Label

fourth_layer_out = graph.get_tensor_by_name("Fully_Connected_Layer_4/Elu:0").op.outputs[0]
"""
2.Create new softmax layer and connect old fourth layer, connect to softmax layer
"""
logits = tf.layers.dense(inputs=fourth_layer_out, # assign fourth layer's output as input
                         units=5, # converge to five nodes
                         kernel_initializer=tf.contrib.layers.variance_scaling_initializer(), # initial with he initializer
                         name = "New_Logits" # name this layer as New_Logists
                        )  # Create the new logits layer using dense

y_ = tf.nn.softmax(logits, name="Y_proba") # Softmax Layer

"""
3. Get the 1th, 2th and logits layer which is what we would like to train
"""
training_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="Fully_Connected_Layer_1") # get softmax layer by scope name
training_layer_vars += tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="Fully_Connected_Layer_2") # get softmax layer by scope name
training_layer_vars += tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="New_Softmax_Layer") # get softmax layer by scope name


"""
4. Define loss, optimizer and assign to only update on softmax_layer which defined above
"""   
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits, name="New_Cross_Entropy"), name="New_Loss") # Refine Loss Function

optimizer = tf.train.AdamOptimizer(0.0017, name="Adam_Op") # create a new optimizer
training_op = optimizer.minimize(loss, var_list=training_layer_vars) # minimize the loss and only update certern variables

"""
5. Define new evaluation metrics
"""
predicted_class = tf.argmax(y_, 1, output_type=tf.int32) # Define Accuracy
correct_predict = tf.equal(y, predicted_class) # [True, False ..., True]
accuracy = tf.reduce_mean(tf.cast(correct_predict, tf.float32)) # [True, False ..., True] --> [1,0,...,1]

"""
Training Preparation
"""

no_update_tolerant = 20 # set the criteria that if the model didn't get better for how many epochs

no_update_progress = 0 # the variables to record for the not better epochs
current_best_accuracy = 0.0 # variables to record the best valid accuracy
current_best_loss = 10.0 # variables to record the best valid loss

saver = tf.train.Saver() # to store the model
with tf.Session() as sess: # start the tensorflow nn session
    sess.run(tf.global_variables_initializer()) # initialize all the parameter
    restore_saver.restore(sess, './Models/Team26_HW2.ckpt') # restore pretrain weights  
    
    """
    Initialize all the variables
    """
    var_list_init = tf.variables_initializer(set(tf.global_variables())-temp)
    sess.run(var_list_init) # init the variable about fourth + softmax layer + adam optimizer
    
    """
    Training Step
    """
    begin_time = time.time() # record the begin training time
    for epochs in range(1000): # training on 1000 epochs
        _, train_loss = sess.run([training_op, loss], feed_dict={X:X_train2, y:y_train2}) # feed with training variables
        valid_loss, valid_accuracy = sess.run([loss, accuracy], feed_dict={X:X_valid2, y:y_valid2}) # feed with validation variables
        test_loss ,  test_accuracy = sess.run([loss, accuracy], feed_dict={X:X_test2, y:y_test2}) # feed with testing variables
        
        if valid_accuracy > current_best_accuracy: # if better
            current_best_accuracy = valid_accuracy # keep record
            save_path = saver.save(sess, "./Models/TransferLearning/HW3_4/Team26_HW3_4.ckpt") #keep the best model
            no_update_progress = 0
        else:                                      # if not better
            no_update_progress += 1                # record on no update

        if valid_loss < current_best_loss: # if better
            current_best_loss = valid_loss # keep record            
            
        if no_update_progress == no_update_tolerant: break # stop training
            
                
        print("{:4d} epochs,   Validation loss:{:.5f},   Best loss:{:.5f},   Accuracy:{:.2f}%/ {:.2f}%".format(
            epochs+1, # show current epochs
            valid_loss, # show validation loss
            current_best_loss, # showe best loss                                                                       
            valid_accuracy*100, # to show as % we need to *100
            test_accuracy*100 # to show as % we need to *100
        )) # print the performane for each epochs
    finish_itme = time.time() # record the finish time
    print("\nTraining Time:{:3f}".format(finish_itme-begin_time)) # calculate and print training time
        
        